In [30]:
import psycopg2
from tabulate import tabulate

In [31]:
print("Beginning")

Beginning


In [32]:
# Connect to your PostgreSQL database
con = psycopg2.connect(
            host="localhost",
            database="demo",
            user="postgres",
            password="Kavya@1997");
print(con)

#For isolation: SERIALIZABLE
con.set_isolation_level(3)
    
# For atomicity
con.autocommit = False

<connection object at 0x00000194AA050D00; dsn: 'user=postgres password=xxx dbname=demo host=localhost', closed: 0>


In [33]:
try:
    # Create a cursor object
    cur = con.cursor()
    print("Connected to the database successfully.")
 
    
except Exception as e:
    print("An error occurred while connecting to the database: {e}")

Connected to the database successfully.


In [34]:
try:
    # Start a transaction for UPDATE and DELETE CASCADE on Stock Table
    cur.execute("BEGIN TRANSACTION")

    # Check if the "Stock" table exists
    cur.execute(
        "SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'stock')"
    )
    stock_exists = cur.fetchone()[0]

    if stock_exists:
        # Define the ALTER TABLE statement
        alter_table_query = ("""
            ALTER TABLE Stock
            ADD CONSTRAINT FK_stock_product FOREIGN KEY (prodid) REFERENCES Product(prodid) ON UPDATE CASCADE ON DELETE CASCADE,
            ADD CONSTRAINT FK_stock_depot FOREIGN KEY (depid) REFERENCES Depot(depid) ON UPDATE CASCADE ON DELETE CASCADE;
        """);

        # Execute the ALTER TABLE statement
        cur.execute(alter_table_query)
        print("ALTER TABLE statement executed successfully.")
    else:
        print("The 'Stock' table does not exist.")

    # Commit the transaction
    con.commit()
except Exception as e:
    # Rollback the transaction if an error occurs
    con.rollback()
    print("Failed to execute ALTER TABLE statement:", e)

Failed to execute ALTER TABLE statement: constraint "fk_stock_product" for relation "stock" already exists



In [37]:
# 1.Define function for first transaction
def delete_product_p1():
    try:
        # Start first transaction
        cur.execute("BEGIN TRANSACTION");

        # Delete the prodid p1 from the Product table
        cur.execute("DELETE FROM Product WHERE prodid='p1'");

        # delete the proid p1 from the Stock table
        cur.execute("DELETE FROM Stock WHERE prodid='p1'");

        # Commit the transaction
        con.commit()
        print("Transaction 1: Product p1 deleted successfully.")
        
    except Exception as e:
         # Rollback the transaction if an error occurs
        con.rollback()
        print("Transaction 1 failed:", e)


In [36]:
# 2.Define function for second transaction
def delete_depot_d1():
    try:
        # Start second transaction
        cur.execute("BEGIN TRANSACTION");

        # Delete the depid d1 from the Depot table
        cur.execute("DELETE FROM Depot WHERE depid = 'd1'");

        # delete the depid d1 from the Stock table
        cur.execute("DELETE FROM Stock WHERE depid = 'd1'");

        # Commit the transaction
        con.commit()
        print("Transaction 2: Depot d1 deleted successfully.")
        
    except Exception as e:
        # Commit the transaction
        con.rollback()
        print("Transaction 2 failed:", e)

In [44]:
# 3.Define function for third transaction
def change_product_name_p1():
    try:
        # Start third transaction
        cur.execute("BEGIN TRANSACTION");

        # Update the prodid p1 to pp1 in the Product table
        cur.execute("UPDATE Product SET prodid = 'pp1' WHERE prodid = 'p1'");

        # Update the prodid p1 to pp1 in the Stock table
        # cur.execute("UPDATE Stock SET prodid = 'pp1' WHERE prodid = 'p1'");
        
        # Commit the transaction
        con.commit()
        print("Transaction 3: Product id changed to pp1 successfully.")

    except Exception as e:
        con.rollback()
        print("Transaction 3 failed:", e)

In [45]:
# 4.Define function for fourth transaction
def change_depot_name_d1():
    try:
        # Start fourth transaction
        cur.execute("BEGIN TRANSACTION");

        # Update the depid d1 to dd1 in the Depot table
        cur.execute("UPDATE Depot SET depid = 'dd1' WHERE depid = 'd1'");

        # Update the depid d1 to dd1 in the Stock table
        # cur.execute("UPDATE Stock SET depid = 'dd1' WHERE depid = 'd1'");

        # Commit the transaction
        con.commit()
        print("Transaction 4: Depot id changed to dd1 successfully.")

    except Exception as e:
        con.rollback()
        print("Transaction 4 failed:", e)  

In [47]:
# 5.Define function for fifth transaction
def add_new_product():
    try:
        # Start fifth transaction
        cur.execute("BEGIN TRANSACTION");
        
        # Adding a product in Product  table
        cur.execute("INSERT INTO Product VALUES ('p100', 'cd', 5)");
        
        # Adding a product in Stock table
        cur.execute("INSERT INTO Stock VALUES ('p100', 'd2', 50)");
        
        # Commit the transaction
        con.commit()
        print("Transaction 5: New product added in Product and Stock successfully.")
        
    except Exception as e:
        # Rollback the transaction if an error occurs
        con.rollback()
        print("Transaction 5 failed:", e)


In [49]:
# 6.Define function for sixth transaction
def add_new_depot():
    try:
        # Start sixth transaction
        cur.execute("BEGIN TRANSACTION");
        
        # Adding a depot in Depot table
        cur.execute("INSERT INTO Depot VALUES ('d100', 'Chicago', 100)");

        # Adding a depot in Stock table
        cur.execute("INSERT INTO Stock VALUES ('p1', 'd100', 100)");

        # Commit the transaction
        con.commit()
        print("Transaction 6: New product added in Depot and Stock successfully.")
        
    except Exception as e:
        # Rollback the transaction if an error occurs
        con.rollback()
        print("Transaction 6 failed:", e)

In [1]:
 # Call each transaction function
delete_product_p1()
delete_depot_d1()
change_product_name_p1()
change_depot_name_d1()
add_new_product()
add_new_depot()

NameError: name 'delete_product_p1' is not defined

In [14]:
# Close the cursor and connection
if con:
        con.commit()
        cur.close
        con.close
        print("PostgreSQL connection is now closed")
        print("End")

PostgreSQL connection is now closed
End
